In [ ]:
# Install selenium itself
!pip install selenium

In [62]:
# Install libraries dependencies
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup
import json
import os
import re
from selenium.webdriver.firefox.options import Options

In [1]:
# Retrieve Answers to Questions within CSV file
def retrieve(expansion, question, subdirectory):
    #create a subdirectory and put the file inside
    subdirectory = subdirectory
    #check if subdirectory exists
    if not os.path.exists(subdirectory):
        os.makedirs(subdirectory)
    # The file is saved within a parent directory that is labelled as your topic name with each file labelled as the question name
    filename = re.sub(r'[<>:"/\\|?*]', '', expansion) + '.jsonl'
    filename = os.path.join(subdirectory, filename)
    
    # Creates a JSONL file and iterates between each answer, depositing it in a dictionary and appending it into the jsonl file
    question_text = question
    print(os.path.exists(filename))
    if os.path.exists(filename):
        print("File", filename, "already exists. Skipping.")
    else:
        with open(filename, 'w', encoding='utf-8') as json_file:
            options = Options()
            # CHANGE THE LINK TO YOUR FIREFOX PROFILE WHERE QUORA IS LOGGED IN + ENABLE COOKIES
            options.profile = ''
            driver = webdriver.Firefox(options=options)
            driver.get("https://www.quora.com/" + expansion.replace(" ","-"))
            time.sleep(5)
            html_source = driver.page_source
            soup_3 = BeautifulSoup(html_source, 'html.parser')
            xpathh = "(//div[contains(@class,'q-box spacing_log_answer_content puppeteer_test_answer_content')])"
            # Find all elements using XPath
            elements = driver.find_elements(By.XPATH, value=xpathh)
            # Iterates between all the answers that were gathered to save within the json_file
            for element in elements:
                inner_html = element.get_attribute('innerHTML')
                soup = BeautifulSoup(inner_html, 'html.parser')
                plain_text = soup.get_text(separator=' ', strip=True)
                #print("Element plain text:", plain_text)
                # Create a dictionary for the question and answer pair
                qa_dict = {"instruction" : question_text, "context" : "" , "responses" : plain_text}
                json_file.write(json.dumps(qa_dict, ensure_ascii=False) + '\n')
            driver.quit()

    # Close the browser
    




In [50]:
#Removes empty files due to error links within the subdirectory
def remove_empty_files(subdirectory):
    # Iterate over files in the subdirectory
    for filename in os.listdir(subdirectory):
        filepath = os.path.join(subdirectory, filename)
        # Check if the file is empty
        if os.path.isfile(filepath) and os.path.getsize(filepath) == 0:
            print("Removing empty file:", filename)
            os.remove(filepath)

In [51]:
#Reads the file and retrieves the list of questions along side with the topic
#Questions CSV is formatted per row as (TOPIC, QUESTIONS, LINK)
with open("questions.csv") as questions:
    next(questions)
    for line in questions:
        line = line.rstrip("\n")
        line = line.split(",")
        expansion = line[1]
        subdirectory = line[0]
        link = line[2].split("/")
        question = link[-1]
        
        #calls the function to find the answer to each question, the directory would be the topic
        retrieve(expansion, question, subdirectory)
    remove_empty_files(subdirectory)
        

True
File Machine Learning\How do I learn machine learning.jsonl already exists. Skipping.
Removing empty file: Could you suggest some good sources for learning machine learning.jsonl
Removing empty file: I have completed Andrew Ng's Coursera class on machine learning. What should I do next What can I do next.jsonl
Removing empty file: I want to learn artificial intelligence and machine learning. Where can I start.jsonl
Removing empty file: Is Andrew Ng's Machine Learning course still the best machine learning course available.jsonl
Removing empty file: Is data sciencemachine learningAI overhyped right now.jsonl
Removing empty file: Is upGrad by IIIT Bangalore (the specialization in AI and machine learning) a good course to be pursued.jsonl
Removing empty file: Machine learning or Data Science.jsonl
Removing empty file: What are the best courses to start learning machine learning I am completely new to it and have some programming background..jsonl
Removing empty file: What are the bes

KeyboardInterrupt: 

In [61]:
#Test run
# This should create open a browser to search for this questions
retrieve("what-is-machine-learning-4", "what is machine learning?", "Machine Learning")

False
